<a href="https://colab.research.google.com/github/Welle-Kim/Python/blob/master/%08%EB%8B%A4%EC%9D%8C_%EC%98%81%ED%99%94%EB%A6%AC%EB%B7%B0_%ED%81%AC%EB%A1%A4%EB%A7%81(10%EA%B0%9C%EC%94%A9).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup as bs
from urllib.parse import quote
import requests
import time
import pandas as pd
import re
import csv
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

from selenium import webdriver
import pandas as pd
import numpy as np

driver = webdriver.Chrome("/opt/homebrew/bin/chromedriver")
# driver = webdriver.Chrome("c:/pydata/chromedriver")
review_data=[]

kakaopage=[122749,154400,158429,131275,152019,152509,142910,114296,137956,147303]


cnt=1
for k in kakaopage:
    driver.get("https://movie.daum.net/moviedb/grade?movieId="+str(k))
    driver.execute_script("window.scrollTo(0, 53)") 

    while True:
        try:
            last_height = driver.execute_script("return document.body.scrollHeight")
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(1)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                xpath1 = """//*[@id="alex-area"]/div/div/div/div[3]/div[1]/button"""
                driver.find_element_by_xpath(xpath1).click()
        except:
            break
        last_height = new_height
    
    html = driver.page_source
    soup=bs(html,"html.parser")
    movie=soup.find("span",class_='txt_tit').text

    reviews=soup.find_all("div",class_="cmt_info")
    
    for review in reviews:
        try:
            sentences=review.find("p",class_='desc_txt font_size_').text.strip()
            sentence=re.sub('[^ ㄱ-ㅣ가-힣+]','',sentences)
        
        except AttributeError as err:
            continue
        
        score=review.find("div").text.strip()
        review_data.append([movie,sentence,score])
        #print(review_data)
        
driver.close()
        
df=pd.DataFrame(review_data,columns=["Movie","Sentence","Score"])
display(df.head(10))
display(df.tail(10))
df.to_csv("./개봉예정_영화관순_1~10순위.csv",header=True,index=False,encoding="utf8")
df.to_excel("./개봉예정_영화관순_1~10순위.xlsx",header=True,index=False)

print("작업이 완료되었습니다.")